Image Prep Phase 3: CSV conversion to TFRECORD (.record)

*Before using this notebook, convert image xml data to csv (Prep Phase 2).

*This prep step creates an .record file for each .csv file.

*Use this notebook to convert a .csv to a single .record file.

*After conversion to .record proceed with image training (Training Phase 1).

*You should have a train_labels.csv and a test_labels.csv in their respective /train and /test folders. 

*The original tutorial runs this code from the command line. This code is adapted to allow you to debug and see what is happening top to bottom. 

**Configured to support training of just a single label. 

**Final Outout: Should be train.record and test.record

References: 

Code unpacked from tutorial at: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

installing tensorflow https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

https://chamaradodandeniya.wordpress.com/2019/04/16/how-to-configure-google-colab-for-object-detection-using-tensorflow/

https://github.com/tensorflow/models/issues/2031

Troubleshooting: 

TensorFlow versioning issues between tutorials and online forums may cause issues. Common problems and solutions:

Will not work: tf.gfile.GFile | Replace with: tf.io.gfile.Gfile

Will not work: tf.python_io.TFRecordWriter | Replace with: tf.io.TFRecordWriter

In [ ]:
import os
import io
import pandas as pd
import tensorflow as tf
#installing tensorflow on mac or pc requires extensive setup, see reference 'installing tensorflow'

import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
#object_detection.utils requires extensive setup, see 'installing tensorflow'

from collections import namedtuple, OrderedDict

In [ ]:
#run once to create a funciton called class_text_to_int, to be called later
#configure the row_label to your label

def class_text_to_int(row_label):
    if row_label == 'homer':  # replace 'homer' with your label
        return 1
    else:
        print('none')
        None

In [ ]:
#run once to create a function called split, to be called later

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [ ]:
#run once to create a function called create_tf_example, to be called later

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    
    filename = group.filename.encode('utf8')
    image_format = b'jpg'

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
#run once with variables set for test and then run again for test; 
#for second run, replace values for csv_input, img_path, and output_path

#full path to your test_labels.csv or train_label.csv file
csv_input = '/images/test/test_labels.csv'

#full path to the location of your images; either train or test
img_path = '/images/test' 

#full path to the output location and .record file
output_path = '/annotations/test.record' 

In [ ]:
#run once, then again after changing variables between test and train

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(os.getcwd(), img_path)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')

In [ ]:
#run once, then again after changing variables between test and train

for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

In [ ]:
#not necessary but print the tfrecord to preview the file that you generated
print(tf_example)